In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip3 install tensorflow_text
!pip install sentencepiece

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout, Lambda, Conv1D, MaxPooling1D, InputLayer, GRU
import time
import csv
import tensorflow_text
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report

In [ ]:
train_data_path = '/content/drive/My Drive/Univer/Diploma/data_tok/train_data_with_vectors.csv'
eval_data_path = '/content/drive/My Drive/Univer/Diploma/data_tok/eval_data_with_vectors.csv'
test_data_path = '/content/drive/My Drive/Univer/Diploma/data_tok/test_data_with_vectors.csv'
train_words_path = '/content/drive/My Drive/Univer/Diploma/data_tok/words_tokens_train.csv'
eval_words_path = '/content/drive/My Drive/Univer/Diploma/data_tok/words_tokens_eval.csv'
test_words_path = '/content/drive/My Drive/Univer/Diploma/data_tok/words_tokens_test.csv'
model_path = '/content/drive/My Drive/Univer/Diploma/data_tok/model_bpe_dip.model'

In [ ]:
train_data = pd.read_csv(train_data_path, sep = ',')
eval_data = pd.read_csv(eval_data_path, sep = ',')
test_data = pd.read_csv(test_data_path, sep = ',')
train_words = pd.read_csv(train_words_path, sep = ',')
eval_words = pd.read_csv(eval_words_path, sep = ',')
test_words = pd.read_csv(test_words_path, sep = ',')

In [ ]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

In [ ]:
train_text_embed_list = []
eval_text_embed_list = []
test_text_embed_list = []

for i, row in train_data.iterrows():
  train_text_embed = embed(row['text'])
  train_text_embed_list.append(train_text_embed)

for i, row in eval_data.iterrows():
  eval_text_embed = embed(row['text'])
  eval_text_embed_list.append(eval_text_embed)

for i, row in test_data.iterrows():
  test_text_embed = embed(row['text'])
  test_text_embed_list.append(test_text_embed)

In [ ]:
MAX_SEQ_LEN = 400

In [ ]:
train_vectors_pad = pad_sequences(train_vectors, maxlen=MAX_SEQ_LEN, padding='post', truncating='post')
eval_vectors_pad = pad_sequences(eval_vectors, maxlen=MAX_SEQ_LEN, padding='post', truncating='post')
test_vectors_pad = pad_sequences(test_vectors, maxlen=MAX_SEQ_LEN, padding='post', truncating='post')

In [ ]:
train_labels = np.array(train_data['label'])
eval_labels = np.array(eval_data['label'])
test_labels = np.array(test_data['label'])

**USE**

In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-multilingual/3"
embedd = hub.KerasLayer(module_url, trainable = True)

In [ ]:
train_embed_arr = np.array(train_text_embed_list)
print(train_embed_arr.shape)

eval_embed_arr = np.array(eval_text_embed_list)
print(eval_embed_arr.shape)

(140, 1, 512)
(123, 1, 512)


In [ ]:
eval_embed_arr_ = np.reshape(eval_embed_arr, (123,512,1))
eval_labels_ = np.reshape(eval_labels, (123,1))

train_embed_arr_ = np.reshape(train_embed_arr, (140,512,1))
train_labels_ = np.reshape(train_labels, (140,1))

In [ ]:
def RNN_GRU(MAX_SEQ_LEN, EMBED_DIM):
    nclasses = 2
    model = Sequential()
    hidden_layer = 3
    gru_node = 32

    model.add(InputLayer(input_shape=(MAX_SEQ_LEN,EMBED_DIM,), dtype='float32'))
    model.add(Dropout(0.5))
    model.add(GRU(gru_node,return_sequences=True, recurrent_dropout=0.2))
    model.add(Dropout(0.5))
    model.add(GRU(gru_node, recurrent_dropout=0.2))
    model.add(Dropout(0.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])
    model.summary
    return model
model_spm_all_rnn = RNN_GRU(MAX_SEQ_LEN = 512, EMBED_DIM = 1)

In [ ]:
model_spm_all_rnn.fit(train_embed_arr_, 
            train_labels_,
            #validation_data=(eval_embed_arr_, eval_labels_),
            epochs=250,
            batch_size=32)

Epoch 1/250
5/5 [==============================] - 8s 683ms/step - loss: 0.6943 - accuracy: 0.4357
Epoch 2/250
5/5 [==============================] - 4s 719ms/step - loss: 0.6933 - accuracy: 0.4500
Epoch 3/250
5/5 [==============================] - 3s 688ms/step - loss: 0.6932 - accuracy: 0.5000
Epoch 4/250
5/5 [==============================] - 4s 712ms/step - loss: 0.6929 - accuracy: 0.5000
Epoch 5/250
5/5 [==============================] - 3s 690ms/step - loss: 0.6920 - accuracy: 0.5143
Epoch 6/250
5/5 [==============================] - 4s 697ms/step - loss: 0.6934 - accuracy: 0.5357
Epoch 7/250
5/5 [==============================] - 3s 692ms/step - loss: 0.6935 - accuracy: 0.5000
Epoch 8/250
5/5 [==============================] - 4s 697ms/step - loss: 0.6929 - accuracy: 0.5500
Epoch 9/250
5/5 [==============================] - 3s 679ms/step - loss: 0.6937 - accuracy: 0.5214
Epoch 10/250
5/5 [==============================] - 4s 716ms/step - loss: 0.6931 - accuracy: 0.5286
Epoch 11/